In [24]:
import numpy as np
import scipy.stats
import pandas as pd
from abby.compare import compare_bootstrap, compare_delta


In [32]:
def generate_dataset(
    num_users=10000,
    baseline_conversion_rate=0.2,
    sessions_skew=0.5,  # controls variance in sessions per user. higher = more variance
    beta_size=1000,  # controls variance in conversion. higher = less variance
    cvr_decay_factor=0.1,
):
    sessions = (
        np.exp(scipy.stats.norm(1, sessions_skew).rvs(num_users)).astype(int)
        + 1
    )

    cvr_func = lambda x, y: x * (
        1 / 10 + 9 / 10 * np.exp(-1 * cvr_decay_factor * (y - 1))
    )
    rates = cvr_func(baseline_conversion_rate, sessions)
    a = rates * beta_size
    b = beta_size - rates * beta_size
    conversion_rates = np.random.beta(a, b, size=num_users)

    conversions = np.random.binomial(
        sessions, conversion_rates, size=num_users
    )
    return sessions, conversions, conversion_rates


def generate_experiment_datasets(sessions, conversions, num_experiments=5000):
    _sessions_ctrl = []
    _sessions_exp = []

    _conversions_ctrl = []
    _conversions_exp = []

    num_users = sessions.shape[0]

    for x in range(0, num_experiments):
        assignments = np.random.choice(num_users, num_users, replace=False)
        control_idxs = assignments[0 : int(num_users / 2)]
        test_idxs = assignments[int(num_users / 2) :]

        _sessions_ctrl.append(sessions[control_idxs])
        _sessions_exp.append(sessions[test_idxs])

        _conversions_ctrl.append(conversions[control_idxs])
        _conversions_exp.append(conversions[test_idxs])

    sessions_ctrl = np.array(_sessions_ctrl).astype(np.float64)
    sessions_exp = np.array(_sessions_exp).astype(np.float64)
    conversions_ctrl = np.array(_conversions_ctrl).astype(np.float64)
    conversions_exp = np.array(_conversions_exp).astype(np.float64)

    return sessions_ctrl, sessions_exp, conversions_ctrl, conversions_exp


In [65]:
sessions_exp, conversions_exp, conversion_rates = generate_dataset(
    baseline_conversion_rate=0.2
)

sessions_ctrl, conversions_ctrl, conversion_rates = generate_dataset(
    baseline_conversion_rate=0.19
)


In [66]:
df = pd.DataFrame()
df["session"] = np.hstack((sessions_ctrl, sessions_exp))
df["conversion"] = np.hstack((conversions_ctrl, conversions_exp))
df["variant_name"] = np.repeat(
    ["control", "experiment"], (len(sessions_ctrl), len(sessions_exp))
)
# df["cvr"] = df.eval("conversion / session")
df.groupby("variant_name")[['conversion', 'session']].sum().assign(ctr = lambda df: df['conversion'] / df['session'])


,conversion,session,ctr
variant_name,,,
control,5056,35631,0.141899
experiment,5377,35476,0.151567


In [67]:
def var_ratio(x, y):  # x/y
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    var_x = np.var(x, ddof=1)
    var_y = np.var(y, ddof=1)
    cov_xy = np.cov(x, y, ddof=1)[0][1]
    result = (
        (
            var_x / mean_x ** 2
            + var_y / mean_y ** 2
            - 2 * cov_xy / (mean_x * mean_y)
        )
        * (mean_x * mean_x)
        / (mean_y * mean_y * len(x))
    )
    return result


# ttest calculation
def ttest(mean_control, mean_treatment, var_control, var_treatment):
    diff = mean_treatment - mean_control
    var = var_control + var_treatment
    stde = 1.96 * np.sqrt(var)
    lower = diff - stde
    upper = diff + stde
    z = diff / np.sqrt(var)
    p_val = stats.norm.sf(abs(z)) * 2

    result = {
        "mean_control": mean_control,
        "mean_experiment": mean_treatment,
        "var_control": var_control,
        "var_experiment": var_treatment,
        "difference": diff,
        "lower_bound": lower,
        "upper_bound": upper,
        "p-value": p_val,
    }
    return pd.DataFrame(result, index=[0])


var_control = var_ratio(conversions_ctrl, sessions_ctrl)
var_treatment = var_ratio(conversions_exp, sessions_exp)
mean_control = conversions_ctrl.sum() / sessions_ctrl.sum()
mean_treatment = conversions_exp.sum() / sessions_exp.sum()

In [68]:
import numpy as np


def ratio_metric_variance(numerator_by_user_array, denominator_by_user_array):
    # in the array, each data points represent one experimentation unit, such as user
    # ratio metric mean numerator_by_user_array / denominator_by_user_array
    X_d_mean = np.mean(denominator_by_user_array)
    X_n_mean = np.mean(numerator_by_user_array)
    X_d_variance = np.var(denominator_by_user_array)
    X_n_variance = np.var(numerator_by_user_array)
    X_d_n_covariance = np.cov(
        denominator_by_user_array, numerator_by_user_array, bias=True
    )[0][1]
    return (
        (X_n_variance) / (X_d_mean ** 2)
        - 2 * X_n_mean * X_d_n_covariance / (X_d_mean ** 3)
        + (X_n_mean ** 2) * (X_d_variance) / (X_d_mean ** 4)
    )


In [69]:
ttest(mean_control, mean_treatment, var_control, var_treatment)

,mean_control,mean_experiment,var_control,var_experiment,difference,lower_bound,upper_bound,p-value
0,0.141899,0.151567,0.000003,0.000004,0.009668,0.004421,0.014916,0.000304


In [70]:
compare_delta(df, ["control", "experiment"], "conversion", "session")

{'mean_control': 0.14189890825404844,
 'mean_experiment': 0.15156725673694893,
 'var_control': 0.03486463015103382,
 'var_experiment': 0.03679871272525411,
 'difference': 0.00966834848290049,
 'lower_bound': -7.0691348025951495,
 'upper_bound': 7.088471499560951,
 'p-value': 0.00030427348742767535,
 'z_score': 3.611634260754107}

In [61]:
stats.norm.ppf(1-0.05/2)

1.959963984540054

In [46]:
compare_bootstrap(
df, ["control", "experiment"], "conversion", "session", n_bootstrap=5_000
)


100%|██████████| 5000/5000 [00:02<00:00, 2207.24it/s]


0.7916000000000001